# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
#csv path
csv_path = "../output_data/WeatherData.csv"

#read in the weather data
weatherdata = pd.read_csv(csv_path)

weatherdata.head()

weatherdata_sorted = weatherdata.sort_values(['Humidity'])
weatherdata_sorted

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
255,Tessalit,20.1986,1.0114,103.23,4,0,5.70,ML,1619366771
6,Arlit,18.7369,7.3853,106.25,7,88,12.19,NE,1619366771
66,Aswan,24.0934,32.9070,97.03,7,0,6.78,EG,1619366946
375,Tanout,14.9709,8.8879,104.16,8,70,9.91,NE,1619366973
468,Kutum,14.2000,24.6667,90.70,9,60,11.23,SD,1619366708
...,...,...,...,...,...,...,...,...,...
384,Sola,-13.8833,167.5500,77.00,100,20,4.85,VU,1619366763
185,Christchurch,-43.5333,172.6333,37.40,100,90,3.44,NZ,1619366971
33,Mount Gambier,-37.8333,140.7667,44.60,100,21,3.40,AU,1619366945
305,Clyde River,70.4692,-68.5914,17.60,100,20,21.85,CA,1619366901


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
#configure gmaps
gmaps.configure(api_key=g_key)

#table with coordinates and humidity
humidity_df= weatherdata[['Lat', 'Lng', 'Humidity']]

#create map
fig = gmaps.figure()

#heatmap layer
heatmap_layer = gmaps.heatmap_layer(humidity_df[['Lat', 'Lng']], weights=humidity_df['Humidity'], 
                                    dissipating = True, max_intensity = 100, point_radius = 20, 
                                    opacity = 0.5)

fig.add_layer(heatmap_layer)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
#isoloate cities with temps between 70-80
under80_df= weatherdata.loc[weatherdata['Max Temp']<80]
ideal_temp = under80_df.loc[under80_df['Max Temp']>70]


#cities with windspeed less than 10 mph

low_wind = ideal_temp.loc[ideal_temp['Wind Speed']<10]


#cities with perfect temp, no wind, and blue skies

blue_sky = low_wind.loc[low_wind['Cloudiness']==0]

ideal_weather = pd.DataFrame(blue_sky)

#reset index
ideal_weather = ideal_weather.reset_index()

ideal_weather

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,144,East London,-33.0153,27.9116,71.60,73,0,8.05,ZA,1619366951
1,277,Santiago,-33.4569,-70.6483,72.00,37,0,2.30,CL,1619366951
2,338,Paphos,34.7667,32.4167,73.99,64,0,1.14,CY,1619366944
3,393,Werda,-25.2667,23.2833,75.02,19,0,2.17,BW,1619366732
4,462,Serowe,-22.3833,26.7167,72.46,21,0,5.12,BW,1619366950
5,476,Villa Carlos Paz,-31.4241,-64.4978,71.01,45,0,4.61,AR,1619366944


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
#new data frame for hotel information
hotel_df = pd.DataFrame(ideal_weather, columns=['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 
                                                'Cloudiness', 'Wind Speed', 'Country', 'Hotel Name'])
hotel_df['Hotel Name']=""

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
0,East London,-33.0153,27.9116,71.60,73,0,8.05,ZA,
1,Santiago,-33.4569,-70.6483,72.00,37,0,2.30,CL,
2,Paphos,34.7667,32.4167,73.99,64,0,1.14,CY,
3,Werda,-25.2667,23.2833,75.02,19,0,2.17,BW,
4,Serowe,-22.3833,26.7167,72.46,21,0,5.12,BW,
5,Villa Carlos Paz,-31.4241,-64.4978,71.01,45,0,4.61,AR,


In [22]:
#find the first hotel for each city located within 5000 meters of the coordinates

#https://maps.googleapis.com/maps/api/place/nearbysearch/json?
#location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY

#return hotel name, city, and country

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {'key': g_key,
          'type': 'lodging',
          'radius': 5000,
          
          
    
}



for index, row in hotel_df.iterrows():
    try:
        params['location']= f"{row['Lat']},{row['Lng']}"

        response = requests.get(base_url , params=params).json()

        hotel_df.loc[index,'Hotel Name']=response['results'][0]['name']
    except:
        print('result not found')

result not found


In [23]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
0,East London,-33.0153,27.9116,71.60,73,0,8.05,ZA,Tu Casa
1,Santiago,-33.4569,-70.6483,72.00,37,0,2.30,CL,Che Lagarto Hostel Santiago
2,Paphos,34.7667,32.4167,73.99,64,0,1.14,CY,Elysium Hotel
3,Werda,-25.2667,23.2833,75.02,19,0,2.17,BW,
4,Serowe,-22.3833,26.7167,72.46,21,0,5.12,BW,Hill View Inn


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [32]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))